# DAT480 Grp 1 pattern matching preprocessing

In [2]:
import numpy as np
from matplotlib import pyplot as plt 
import subprocess
import os
from os import listdir
from os.path import isfile, join
import re
import string
import random
from tqdm import tqdm

## Initalize files

In [157]:
path = "/home/jonathan/Documents/Chalmers/Year5/DAT480/Lab_Project/DAT480/Rules_div_by_length/"
r_hashed = open("/home/jonathan/Documents/Chalmers/Year5/DAT480/Lab_Project/pattern_match_snort3_content_hashed.txt")
r_length = open("/home/jonathan/Documents/Chalmers/Year5/DAT480/Lab_Project/pattern_match_snort3_content_length.txt")
r_text = open("/home/jonathan/Documents/Chalmers/Year5/DAT480/Lab_Project/pattern_match_snort3_content.txt")
t_hashed = r_hashed.read().splitlines(False)
t_hashed = np.array(t_hashed, dtype = str)
t_length = r_length.read().splitlines(False)
t_length = np.array(t_length, dtype=int)
t_text = r_text.read().splitlines(False)
t_text = np.array(t_text, dtype=str)
r_hashed.close()
r_length.close()

rules_hashed = np.array([t_length, t_hashed, t_text]).T
rules_len_sort = rules_hashed[np.argsort(t_length,axis=0)]

## Create generic arrays to use for HLS

In [148]:
arr = (np.array(np.unique(rules_len_sort[:,0],return_counts=True),dtype=int))
indices=np.argsort(arr[0])

sorted_arr = np.array([arr[0][indices], arr[1][indices]])

#write the number of elements per unique length in the order of the lengths
f = open(path+"elements.h","w+")
f.write("const uint16_t elements [] = {\n")
for k in range(len(sorted_arr[0][0:5])):
    f.write(str(sorted_arr[1][k+3])+",\n")
f.write("};")

#write the unique lengths in order
f = open(path+"lengths.h","w+")
f.write("const uint16_t lengths [] = {\n")
for k in sorted_arr[0][3:8]:
    f.write(str(k)+",\n")
f.write("};")
f.close()    

breakif = (sorted_arr[1][3:8]).sum()
start_at = (sorted_arr[1][0:3]).sum()
f = open(path+"ruleset.h", "w+")
f.write("const uint32_t rules [] = {\n")
for i in range(breakif):
    f.write("0x"+str(rules_len_sort[i+start_at][1])+",\n")
f.write("};")
f.close()

f = open(path+"str_ruleset.h", "w+")
curr_len = 0
for length in sorted_arr[0][3:8]:
    f.write("const char str_rules_"+str(length)+ "[]["+str(length)+"] = {\n")
    for el in rules_len_sort[rules_len_sort[:,0] == str(length)]:
        f.write("{")
        for byte in el[2]:
            f.write("'"+str(byte)+"',")
        f.write("},\n")
    f.write("};\n")
f.close()



In [147]:
sorted_arr[0][3:8]

array([4, 5, 6, 7, 8])

In [131]:
rules_len_sort[rules_len_sort[:,0] == '1']

array([['1', 'f4dbdf21', '0'],
       ['1', '65bfb3b6', '~'],
       ['1', '83dcefb7', '1'],
       ['1', '7f6567cb', 'j'],
       ['1', '1363f226', '>'],
       ['1', '79d3d2d4', '/'],
       ['1', 'd3d99e8b', 'A'],
       ['1', 'ed4e242', '.'],
       ['1', 'a4deae1d', '@'],
       ['1', '99063bca', '%'],
       ['1', 'e96ccf45', ' ']], dtype='<U364')

## Create test vectors for the testbench, i.e generate input reference and golden reference

In [167]:
number_of_rules_to_test = 100
number = 0
lengths = sorted_arr[0]
hash_bin = "/home/jonathan/Documents/Chalmers/Year5/DAT480/Prototyping/CRC/hash.o" #precompiled binary to compute the hashes
input_vec = open(path+"test_vec_input.txt","w+")
gold_vec = open(path+"test_vec_gold.txt","w+")
input_str = ""
#mix randomness with rules from the ruleset
for i in range(1,number_of_rules_to_test*2):
    if(i%3 != 0):
        length = random.randrange(1, 20)
        randomstr = ''.join(random.choices(string.ascii_letters+string.digits,k=length))
        input_vec.write(randomstr)
        input_str += randomstr

    else:
        number = random.randrange(start_at,breakif)
        text = rules_len_sort[number][2]
        input_str += text 
        input_vec.write(text)
        
input_vec.close()
for i in tqdm(range(len(input_str))):
    max_length = -1
    curr_element = []
    for length in lengths[3:8]:
        args = (hash_bin,input_str[i:i+length])
        popen = subprocess.Popen(args, stdout=subprocess.PIPE)
        popen.wait()
        output = popen.stdout.read().decode().strip("\n")
        res = np.where(rules_len_sort[(rules_len_sort[:,0].astype(int) >= 4) & (rules_len_sort[:,0].astype(int) <9)] == np.char.lower(output))[0] #efficiency ftw
        if(res.size != 0 and max_length < length):
            max_length = length
            curr_element = res[0]
    if(curr_element):
        gold_vec.write(str(curr_element)+"\n")

gold_vec.close()

100%|██████████| 1761/1761 [02:05<00:00, 14.04it/s]


In [165]:
rules_len_sort[(rules_len_sort[:,0].astype(int) >= 4) & (rules_len_sort[:,0].astype(int) <9)]

array(['1', '1', '1', ..., '240', '262', '364'], dtype='<U364')